In [1]:
from eda import *

In [2]:
ibis.options.interactive = True

In [3]:
downloads.filter(ibis._.timestamp > datetime.now() - timedelta(days=30)).group_by(
    ibis._.system
).agg(ibis._.downloads.sum().name("downloads")).order_by(ibis._.downloads.desc())

┏━━━━━━━━━┳━━━━━━━━━━━┓
┃ system  ┃ downloads ┃
┡━━━━━━━━━╇━━━━━━━━━━━┩
│ string  │ int64     │
├─────────┼───────────┤
│ Linux   │     60981 │
│ NULL    │     17231 │
│ Windows │       498 │
│ Darwin  │       356 │
└─────────┴───────────┘

In [4]:
t = (
    downloads.group_by(
        [ibis._.timestamp.truncate("M").name("timestamp"), ibis._.version]
    )
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), group_by="version", order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [5]:
t

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ timestamp           ┃ version      ┃ downloads ┃ total_downloads ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ timestamp           │ string       │ int64     │ int64           │
├─────────────────────┼──────────────┼───────────┼─────────────────┤
│ 2023-07-01 00:00:00 │ 0.12.0       │        35 │            7738 │
│ 2023-07-01 00:00:00 │ 5.0.0        │      4305 │           45024 │
│ 2023-07-01 00:00:00 │ 5.1.0        │      6689 │           32998 │
│ 2023-07-01 00:00:00 │ 5.1.0.dev465 │         1 │             117 │
│ 2023-07-01 00:00:00 │ 0.4.1        │        12 │            4237 │
│ 2023-07-01 00:00:00 │ 6.0.1.dev33  │       147 │             147 │
│ 2023-07-01 00:00:00 │ 1.0.0        │        20 │           36963 │
│ 2023-07-01 00:00:00 │ 3.0.1        │        23 │            1431 │
│ 2023-07-01 00:00:00 │ v0.6.0       │        15 │            4557 │
│ 2023-07-01 00:00:00 │ 0.3.0        │        16 │            5901 │
│ …                   │ …            │         … │               … │
└─────────────────────┴──────────────┴───────────┴─────────────────┘

In [6]:
fig = px.line(
    t.to_pandas(),
    x="timestamp",
    y="total_downloads",
    color="version",
    title="Downloads",
)

In [7]:
fig

In [8]:
t_mod = clean_data(ibis.read_parquet("data/pypi/modin/*.parquet"))
t_mod = t_mod.drop("project").unpack("file").unpack("details")
t_mod = (
    t_mod.group_by(
        [
            ibis._.timestamp.truncate("D").name("timestamp"),
            ibis._.country_code,
            ibis._.version,
            ibis._.python,
            ibis._.system["name"].name("system"),
        ]
    )
    .agg(
        ibis._.count().name("downloads"),
    )
    .order_by(ibis._.timestamp.desc())
    .mutate(
        total_downloads=ibis._.downloads.sum().over(
            rows=(0, None),
            group_by=["country_code", "version", "python", "system"],
            order_by=ibis._.timestamp.desc(),
        )
    )
    .order_by(ibis._.timestamp.desc())
)

In [9]:
t_pola = clean_data(ibis.read_parquet("data/pypi/polars/*.parquet"))
t_pola = t_pola.drop("project").unpack("file").unpack("details")
t_pola = (
    t_pola.group_by(
        [
            ibis._.timestamp.truncate("D").name("timestamp"),
            ibis._.country_code,
            ibis._.version,
            ibis._.python,
            ibis._.system["name"].name("system"),
        ]
    )
    .agg(
        ibis._.count().name("downloads"),
    )
    .order_by(ibis._.timestamp.desc())
    .mutate(
        total_downloads=ibis._.downloads.sum().over(
            rows=(0, None),
            group_by=["country_code", "version", "python", "system"],
            order_by=ibis._.timestamp.desc(),
        )
    )
    .order_by(ibis._.timestamp.desc())
)

In [10]:
t_pola.schema()

ibis.Schema {
  timestamp        timestamp
  country_code     string
  version          string
  python           string
  system           string
  downloads        int64
  total_downloads  int64
}

In [11]:
t = (
    t_pola.group_by(
        [ibis._.timestamp.truncate("M").name("timestamp"), ibis._.version]
    )
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), group_by="version", order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [12]:
fig = px.line(
    t.to_pandas(),
    x="timestamp",
    y="total_downloads",
    color="version",
    title="Downloads",
)

In [13]:
fig

In [14]:
t = (
    t_mod.group_by(
        [ibis._.timestamp.truncate("M").name("timestamp"), ibis._.version]
    )
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), group_by="version", order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [15]:
fig = px.line(
    t.to_pandas(),
    x="timestamp",
    y="total_downloads",
    color="version",
    title="Downloads",
)

In [16]:
fig

In [17]:
t = (
    downloads.group_by(
        [ibis._.timestamp.truncate("W").name("timestamp"), ibis._.version]
    )
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), group_by="version", order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [18]:
fig = px.line(
    t.to_pandas(), x="timestamp", y="downloads", color="version", title="Downloads"
)

In [19]:
fig

In [20]:
t = (
    downloads.group_by([ibis._.timestamp.truncate("M").name("timestamp")])
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [21]:
fig = px.line(t.to_pandas(), x="timestamp", y="downloads", title="Downloads")

In [22]:
fig

In [35]:
t = (
    t_mod.group_by([ibis._.timestamp.truncate("M").name("timestamp")])
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [36]:
fig = px.line(t.to_pandas(), x="timestamp", y="downloads", title="Downloads")

In [37]:
fig

In [38]:
t = (
    t_pola.group_by([ibis._.timestamp.truncate("M").name("timestamp")])
    .agg(ibis._.downloads.sum().name("downloads"))
    .order_by(ibis._.timestamp.desc())
    .mutate(
        ibis._.downloads.sum()
        .over(rows=(0, None), order_by=ibis._.timestamp.desc())
        .name("total_downloads")
    )
    .order_by(ibis._.timestamp.desc())
)

In [39]:
fig = px.line(t.to_pandas(), x="timestamp", y="downloads", title="Downloads")

In [40]:
fig

In [23]:
fig = px.line(stars.to_pandas(), x="starred_at", y="total_stars", title="Stars")

In [24]:
fig

In [25]:
fig = px.line(pulls.to_pandas(), x="created_at", y="total_pulls", title="Pulls")

In [26]:
fig

In [27]:
downloads

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ timestamp           ┃ country_code ┃ version      ┃ python  ┃ system ┃ downloads ┃ total_downloads ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ timestamp           │ string       │ string       │ string  │ string │ int64     │ int64           │
├─────────────────────┼──────────────┼──────────────┼─────────┼────────┼───────────┼─────────────────┤
│ 2023-07-14 00:00:00 │ US           │ 5.1.0        │ 3.8.17  │ Linux  │        14 │             335 │
│ 2023-07-14 00:00:00 │ NL           │ 5.1.0        │ NULL    │ NULL   │         1 │              84 │
│ 2023-07-14 00:00:00 │ US           │ 1.3.0        │ 3.10.12 │ Linux  │         2 │               2 │
│ 2023-07-14 00:00:00 │ US           │ 5.1.0        │ 3.11.3  │ Darwin │         1 │              15 │
│ 2023-07-14 00:00:00 │ US           │ 5.1.1.dev596 │ NULL    │ NULL   │         2 │              64 │
│ 2023-07-14 00:00:00 │ US           │ 6.0.0        │ 3.10.11 │ Linux  │         3 │              26 │
│ 2023-07-14 00:00:00 │ US           │ 6.0.0        │ 3.10.5  │ Linux  │        17 │             134 │
│ 2023-07-14 00:00:00 │ GB           │ 2.1.1        │ NULL    │ NULL   │         1 │             198 │
│ 2023-07-14 00:00:00 │ US           │ 1.4.0        │ NULL    │ NULL   │         3 │            8376 │
│ 2023-07-14 00:00:00 │ US           │ 3.0.2        │ 3.10.12 │ Linux  │         1 │               7 │
│ …                   │ …            │ …            │ …       │ …      │         … │               … │
└─────────────────────┴──────────────┴──────────────┴─────────┴────────┴───────────┴─────────────────┘

In [28]:
downloads.filter(ibis._.timestamp > datetime.now() - timedelta(days=7)).order_by(ibis._.downloads.desc())

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ timestamp           ┃ country_code ┃ version ┃ python  ┃ system ┃ downloads ┃ total_downloads ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ timestamp           │ string       │ string  │ string  │ string │ int64     │ int64           │
├─────────────────────┼──────────────┼─────────┼─────────┼────────┼───────────┼─────────────────┤
│ 2023-07-11 00:00:00 │ US           │ 1.3.0   │ 3.7.7   │ Linux  │      1212 │         2568200 │
│ 2023-07-12 00:00:00 │ US           │ 6.0.0   │ 3.10.10 │ Linux  │       616 │            1300 │
│ 2023-07-12 00:00:00 │ US           │ 5.1.0   │ 3.10.10 │ Linux  │       614 │            3611 │
│ 2023-07-09 00:00:00 │ IL           │ 5.0.0   │ 3.10.9  │ Linux  │       459 │           15169 │
│ 2023-07-10 00:00:00 │ US           │ 1.3.0   │ 3.7.7   │ Linux  │       443 │         2566988 │
│ 2023-07-14 00:00:00 │ US           │ 1.3.0   │ 3.7.7   │ Linux  │       436 │         2569310 │
│ 2023-07-13 00:00:00 │ US           │ 1.3.0   │ 3.7.7   │ Linux  │       416 │         2568874 │
│ 2023-07-12 00:00:00 │ US           │ 5.1.0   │ 3.9.15  │ Linux  │       412 │            5317 │
│ 2023-07-11 00:00:00 │ US           │ 2.1.1   │ NULL    │ NULL   │       404 │          280077 │
│ 2023-07-13 00:00:00 │ US           │ 2.0.0   │ 3.9.5   │ Linux  │       379 │           26026 │
│ …                   │ …            │ …       │ …       │ …      │         … │               … │
└─────────────────────┴──────────────┴─────────┴─────────┴────────┴───────────┴─────────────────┘

In [29]:
print(
    f"""
    Total downloads: {downloads.downloads.sum().to_pandas():,}
    Total stars: {stars.count().to_pandas():,}
    Total pulls: {pulls.count().to_pandas():,}
    """
)


    Total downloads: 9,667,015
    Total stars: 2,891
    Total pulls: 4,387
    


In [30]:
import pandas as pd

In [31]:
downloads = pd.read_parquet("data/pypi/polars/file_downloads.parquet")

In [32]:
downloads

,timestamp,country_code,url,project,file,details,tls_protocol,tls_cipher
0,2023-03-10 12:56:48+00:00,US,/packages/48/7b/30ac9d95b9d2d1d5dd8d4e811ef71a...,polars,"{'filename': 'polars-0.12.5.tar.gz', 'project'...","{'installer': {'name': 'pip', 'version': '21.3...",TLSv1.2,ECDHE-RSA-AES128-GCM-SHA256
1,2023-03-10 14:38:37+00:00,IE,/packages/b5/ab/ce1463d502788b42453282ebbe89f7...,polars,{'filename': 'polars-0.15.15-cp37-abi3-manylin...,"{'installer': {'name': 'pip', 'version': '22.3...",TLSv1.2,ECDHE-RSA-AES128-GCM-SHA256
2,2021-06-09 18:25:21+00:00,US,/packages/42/ee/5fb4d9f722dfd72c3bfb59bc3beedc...,polars,{'filename': 'polars-0.8.4-cp36-abi3-manylinux...,"{'installer': {'name': 'pip', 'version': '21.1...",TLSv1.2,ECDHE-RSA-AES128-GCM-SHA256
3,2023-05-28 19:24:22+00:00,US,/packages/46/9f/06c45b6c8da613608e3e1c636da00f...,polars,{'filename': 'polars-0.17.15-cp37-abi3-manylin...,"{'installer': {'name': 'pip', 'version': '23.1...",TLSv1.2,ECDHE-RSA-AES128-GCM-SHA256
4,2022-10-19 07:54:58+00:00,NL,/packages/9c/9f/b225850e1db31067817450b0f32bb3...,polars,{'filename': 'polars-0.13.62-cp37-abi3-manylin...,"{'installer': {'name': 'pip', 'version': '22.3...",TLSv1.2,ECDHE-RSA-AES128-GCM-SHA256
...,...,...,...,...,...,...,...,...
5236476,2022-07-27 06:02:30+00:00,IE,/packages/75/e2/294e0d812cc0dff689713d581c94b1...,polars,{'filename': 'polars-0.13.29-cp37-abi3-manylin...,"{'installer': {'name': 'pip', 'version': '20.2...",TLSv1.2,ECDHE-RSA-AES128-GCM-SHA256
5236477,2022-07-29 01:22:19+00:00,IE,/packages/75/e2/294e0d812cc0dff689713d581c94b1...,polars,{'filename': 'polars-0.13.29-cp37-abi3-manylin...,"{'installer': {'name': 'pip', 'version': '20.2...",TLSv1.2,ECDHE-RSA-AES128-GCM-SHA256
5236478,2022-07-29 00:02:51+00:00,IE,/packages/75/e2/294e0d812cc0dff689713d581c94b1...,polars,{'filename': 'polars-0.13.29-cp37-abi3-manylin...,"{'installer': {'name': 'pip', 'version': '22.1...",TLSv1.2,ECDHE-RSA-AES128-GCM-SHA256
5236479,2022-07-29 23:11:52+00:00,IE,/packages/75/e2/294e0d812cc0dff689713d581c94b1...,polars,{'filename': 'polars-0.13.29-cp37-abi3-manylin...,"{'installer': {'name': 'pip', 'version': '22.1...",TLSv1.2,ECDHE-RSA-AES128-GCM-SHA256


In [33]:
downloads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5236481 entries, 0 to 5236480
Data columns (total 8 columns):
 #   Column        Dtype              
---  ------        -----              
 0   timestamp     datetime64[ns, UTC]
 1   country_code  object             
 2   url           object             
 3   project       object             
 4   file          object             
 5   details       object             
 6   tls_protocol  object             
 7   tls_cipher    object             
dtypes: datetime64[ns, UTC](1), object(7)
memory usage: 319.6+ MB


In [34]:
downloads.groupby("country_code").count().sort_values("file", ascending=False)

,timestamp,url,project,file,details,tls_protocol,tls_cipher
country_code,,,,,,,
US,2846565,2846565,2846565,2846565,2723533,2846565,2846565
IE,975455,975455,975455,975455,973830,975455,975455
NL,174316,174316,174316,174316,172052,174316,174316
DE,159095,159095,159095,159095,149372,159095,159095
CN,119106,119106,119106,119106,118991,119106,119106
...,...,...,...,...,...,...,...
DJ,1,1,1,1,1,1,1
LI,1,1,1,1,0,1,1
SL,1,1,1,1,1,1,1
